In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importa as bibliotecas
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import plot_confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#Lê arquivos csv
df = pd.read_csv('../input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('../input/costa-rican-household-poverty-prediction/test.csv')

In [ ]:
#Verifica nulos
df_nan = df.isna().sum()[df.isna().sum() >0]
df_nan

In [ ]:
#Função para imputar valores NA 
def imput_na(df):
    df['v2a1'] = df['v2a1'].fillna(df['v2a1'].median())
    df['v18q1'] = df['v18q1'].fillna(-1)
    df['rez_esc'] = df['rez_esc'].fillna(-1)
    df['meaneduc'] = df['meaneduc'].fillna(df['meaneduc'].median())
    df['SQBmeaned'] = df['SQBmeaned'].fillna(df['SQBmeaned'].median())
    return df

In [ ]:
#Substitui yes e no por 1 e 0
# yes = 1 
# no  = 0
def yes_no(df, column, tipo): 
    map_yes_no = {'yes': 1, 'no': 0}
    df[column] = df[column].replace(map_yes_no).astype(tipo)
    return(df)

In [ ]:
#Imputa
df = imput_na(df)
##
#Substitui yes e no
df = yes_no(df, 'edjefa', int)
df = yes_no(df, 'edjefe', int)
df = yes_no(df, 'dependency', float)

In [ ]:
#Visualiza features do tipo object
df.select_dtypes('object').head()

In [ ]:
#Verifica dados em dependency
df['dependency'].unique()

In [ ]:
#Verifica dados em edjefe
df['edjefe'].unique()

In [ ]:
#Verifica dados em edjefa
df['edjefe'].unique()

In [ ]:
df.info()

In [ ]:
feats = list(df.select_dtypes(['float','int']).columns)
## Identificando variáveis
inv = []
binary = []
cat = []
quant = []
for feat in feats:
    if df[feat].nunique() == 1:    #Se tiver somente um valor se encaixa em inválida pois não agrega nada
        inv.append(feat)
    elif df[feat].nunique() == 2:  #Se tiver somente dois valores vai pra lista de binários
        binary.append(feat)
    elif df[feat].nunique() < 16:  #Se tiver até 15 valores distintos será interpretada como categórica
        cat.append(feat)
    else:
        quant.append(feat)         #Se não for nenhuma das anteriores será interpretada como quantitativa

In [ ]:
#Printa / plota resultado da avaliação das features (informativo)
##-----------------------------------------------------
##- Features com um único valor
if len(inv) > 0:
    for feat in inv:
        print('Feature inválida:', feat)
##-----------------------------------------------------
##- Binárias
#Verifica quantidade para adequar linhas e colunas:
print('***********************************')
print('Quantidade de binárias     :', len(binary))
print('Quantidade de categóricas  :', len(cat))
print('Quantidade de quantitativas:', len(quant))
print('***********************************')

In [ ]:
## Histograma de variáveis discretas
ax = df[quant].hist(bins=10, grid=True, figsize=(20,20), rwidth=0.8)
plt.suptitle("Distribuição de features Quantitativas", fontsize = 12)
plt.show();

In [ ]:
# Histograma da Variável Target
#Agrupa por valor
tar = (
    pd.DataFrame(df.groupby('Target').size())
    .reset_index()
    .rename(columns={0:'quantidade'})
)
tar['Target'] = tar.Target.astype(str)
#Plota
plt.figure(figsize=(8,5))
plt.grid(linestyle=':', linewidth=0.7)
plt.bar(tar.Target,tar.quantidade)
plt.show();

In [ ]:
#Verifica os valores
tar['percent'] = round((tar.quantidade/tar.quantidade.sum())*100,2)
tar

Como pode ser observado existe um problema de desbalanceamento de classes, a categoria 4 ocupa quase 63% do dataset - Over-sampling

In [ ]:
#Separa features em target e explicativas
X = df.drop(['Id','Target', 'idhogar','elimbasu5'], axis=1) #elimbasu5 retirada pois tem o mesmo valor para todos os registros
y = df['Target']
#Separa em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Fazendo o over-sampling
ros = RandomOverSampler(random_state=42)
X_ros,y_ros= ros.fit_resample(X_train,y_train)
# Verificando o resultado
y_ros.value_counts()

XGBoost

In [ ]:
#Instancia xgb
xgbc = XGBClassifier(n_estimators=100, learning_rate=1, random_state=42, max_depth=1)
# Treinando o modelo xgbc
xgbc.fit(X_ros, y_ros);

Random Forest

In [ ]:
# Instanciando o random forest classifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)
# Treinando rfc
rfc.fit(X_ros, y_ros);

GBM

In [ ]:
# Instanciando GBM
gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=0.9, random_state=42, max_depth=1)
gbm.fit(X_ros, y_ros)

In [ ]:
#Visualizar a importância das features
#---------------------------------------------------------------
#Prepara vizualização coeficientes xgbc
xgbc_importance = (
    pd.DataFrame(xgbc.feature_importances_, X_ros.columns)
    .rename(columns={0:'valor'})
    .sort_values('valor')
    .reset_index()  
)
#
#---------------------------------------------------------------
#Prepara vizualização coeficientes rfc
rfc_importance = (
    pd.DataFrame(rfc.feature_importances_, X_ros.columns)
    .rename(columns={0:'valor'})
    .sort_values('valor')
    .reset_index()  
)
#
#---------------------------------------------------------------
#Prepara vizualização coeficientes gbm
gbm_importance = (
    pd.DataFrame(gbm.feature_importances_, X_ros.columns)
    .rename(columns={0:'valor'})
    .sort_values('valor')
    .reset_index()  
)
#---------------------------------------------------------------
#plota
fig, ax =  plt.subplots(nrows=1,ncols=3, figsize=(24,26))
y_pos_xgbc = np.arange(len(xgbc_importance['index']))
y_pos_rfc = np.arange(len(rfc_importance['index']))
y_pos_gbm = np.arange(len(gbm_importance['index']))
#
ax[0].barh(xgbc_importance['index'], xgbc_importance['valor'])
ax[0].set_ylim(min(y_pos_xgbc)-1, max(y_pos_xgbc)+1)
ax[0].title.set_text('XGBClassifier')
#
ax[1].barh(rfc_importance['index'], rfc_importance['valor'])
ax[1].set_ylim(min(y_pos_rfc)-1, max(y_pos_rfc)+1)
ax[1].title.set_text('RFClassifier')
#
ax[2].barh(gbm_importance['index'], gbm_importance['valor'])
ax[2].set_ylim(min(y_pos_gbm)-1, max(y_pos_gbm)+1)
ax[2].title.set_text('GBM')
#---------------------------------------------------------------
plt.show();

In [ ]:
#Visualiza acurácia
predict_xgbc = xgbc.predict(X_test)
accuracy_xgbc = accuracy_score(y_test, predict_xgbc) * 100
#
predict_rfc = rfc.predict(X_test)
accuracy_rfc = accuracy_score(y_test, predict_rfc) * 100
#
predict_gbm = gbm.predict(X_test)
accuracy_gbm = accuracy_score(y_test, predict_gbm) * 100
#
print('Acurácia XGBClassifier:', accuracy_xgbc)
print('Acurácia RFClassifier :', accuracy_rfc)
print('Acurácia GBM          :', accuracy_gbm)

In [ ]:
#Matriz de confusão xgbc
plot_confusion_matrix(xgbc,X_test,y_test);

In [ ]:
#Matriz de confusão random forest
plot_confusion_matrix(rfc,X_test,y_test);

In [ ]:
#Matriz de confusão gbm
plot_confusion_matrix(gbm,X_test,y_test);

Todo tratamento feito nas features explicativas de treino deve ser feito nas features de teste. 

Relatórios e verificações não serão feitas

In [ ]:
#Imputa
test = imput_na(test)
##
#Substitui yes e no
test = yes_no(test, 'edjefa', int)
test = yes_no(test, 'edjefe', int)
test = yes_no(test, 'dependency', float)

In [ ]:
#Separa as features explicativas
feats_exp = test.drop(['Id','idhogar','elimbasu5'], axis=1) #elimbasu5 retirada pois tem o mesmo valor para todos os registros e Target não existe neste dataset

In [ ]:
# Prever o Target de teste usando o modelo xgbc
#test['Target'] = xgbc.predict(feats_exp).astype(int)

In [ ]:
# Prever o Target de teste usando o modelo rfc
#test['Target'] = rfc.predict(feats_exp).astype(int)

In [ ]:
# Prever o Target de teste usando o modelo gbm
test['Target'] = gbm.predict(feats_exp).astype(int)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)